## Objet

L'objet de ce notebook est de contrôler la remarque faite par Arnaud Lafourcade concernant l'évolution de la marge €/kg qui lui semblait élevée.

## Imports et setup technique

In [1]:
from pathlib import Path
import sys
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from importlib import reload
project_root = str(Path(sys.path[0]).parents[0].absolute())
project_root
if project_root not in sys.path:
    sys.path.append(project_root)
import multiprocessing as mp
    
from scripts.utils import process_df  # traitement des avoirs
    
data_path = Path('..') / 'data'
persist_path = Path('..') / 'persist'
from IPython.display import display, HTML
display(HTML("<style>.container { width:90%; }</style>"))

from dask.distributed import LocalCluster, Client
import dask.dataframe as dd
# client = Client(
#     n_workers=int(0.9 * mp.cpu_count()),
#     processes=True,
#     threads_per_worker=1,
#     dashboard_address=':35000'
# ) # n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB'
# client

Client Scheduler: tcp://127.0.0.1:38234 Dashboard: http://127.0.0.1:35000/status,Cluster Workers: 7 Cores: 7 Memory: 33.56 GB


## Chargement des données

### Historiques de vente

In [14]:
%%time
with LocalCluster(
    n_workers=int(0.9 * mp.cpu_count()),
    processes=True,
    threads_per_worker=1,
    dashboard_address=':36000',
#     memory_limit='2GB',
#     ip='tcp://localhost:9895',
) as cluster, Client(cluster) as client_:
    raw_data = dd.read_parquet(persist_path / 'raw_data.parquet')
    print(f'Il y a {len(raw_data)} lignes dans le dataset')

Il y a 229941217 lignes dans le dataset
CPU times: user 1.13 s, sys: 262 ms, total: 1.39 s
Wall time: 4.74 s


In [15]:
for field in raw_data.dtypes.loc[lambda x: x == 'category'].index:
    raw_data[field] = raw_data[field].cat.set_categories(raw_data[field].head(1).cat.categories)

### Référentiel client

In [4]:
df_clt = pd.read_pickle(persist_path / 'clt.pkl')

### Définition des périodes

In [28]:
periods = {
    '20190102': ('2019-01-01', '2019-02-28'),
    '20200102': ('2020-01-01', '2020-02-29'),
}

In [32]:
ddf_list = [raw_data.loc[period[0]:period[1]] for period_key, period in periods.items()]
filter1 = dd.concat(ddf_list, axis=0, keys=periods.keys())

In [27]:
%%time
with LocalCluster(
    n_workers=int(0.9 * mp.cpu_count()),
    processes=True,
    threads_per_worker=1,
    dashboard_address=':36000',
#     memory_limit='2GB',
#     ip='tcp://localhost:9895',
) as cluster, Client(cluster) as client_:
    display(raw_data.loc[periods['20190102'][0]:periods['20190102'][1]].compute().head(10))

,orgacom,month,week,pricetype_init,pricetype_applied,mercu_init,mercu_applied,client,doctype,origin,...,material,brutrevenue,brutrevcur,netrevenue,netrevcur,weight,weightunit,margin,margincur,marginperkg
date,,,,,,,,,,,,,,,,,,,,,
2019-01-01,1RAA,201901,NaN,ZTG,ZTM,H,H,0000222503,ZC10,VR,...,000000000000031753,16.75,EUR,16.75,EUR,2.815,KG,3.83,EUR,1.36
2019-01-01,1RAA,201901,NaN,ZTG,ZTM,H,H,0000222503,ZC10,VR,...,000000000000030859,16.60,EUR,16.60,EUR,2.790,KG,3.88,EUR,1.39
2019-01-01,1RAA,201901,NaN,ZTG,ZTM,H,H,0000222503,ZC10,VR,...,000000000000087033,38.14,EUR,38.14,EUR,4.015,KG,7.95,EUR,1.98
2019-01-01,1RAA,201901,NaN,ZTG,ZTM,H,H,0000221929,ZC10,VR,...,000000000000185145,7.95,EUR,7.95,EUR,0.700,KG,2.57,EUR,3.67
2019-01-01,1RAA,201901,NaN,ZTG,ZTM,H,H,0000222503,ZC10,VR,...,000000000000183130,24.05,EUR,24.05,EUR,1.724,KG,4.25,EUR,2.47
2019-01-01,1RAA,201901,NaN,ZTG,ZTM,H,H,0000221929,ZC10,VR,...,000000000000185044,8.50,EUR,8.50,EUR,0.950,KG,2.68,EUR,2.82
2019-01-01,1RAA,201901,NaN,ZTG,ZTM,H,H,0000222503,ZC10,VR,...,000000000000177429,71.64,EUR,71.64,EUR,17.820,KG,13.05,EUR,0.73
2019-01-01,1RAA,201901,NaN,ZTG,ZTM,H,H,0000222503,ZC10,VR,...,000000000000002477,12.24,EUR,12.24,EUR,18.576,KG,3.73,EUR,0.20
2019-01-01,1RAA,201901,NaN,ZTG,ZTM,H,H,0000222640,ZC10,TV,...,000000000000035487,36.00,EUR,36.00,EUR,5.000,KG,14.37,EUR,2.87


CPU times: user 10.3 s, sys: 7.19 s, total: 17.5 s
Wall time: 23.1 s


In [40]:
%%time
ddf_list[0].orgacom.cat.ordered

CPU times: user 611 µs, sys: 0 ns, total: 611 µs
Wall time: 621 µs


False